In [ ]:
import matplotlib.pyplot as plt

# Data for sentiment analysis metrics visualization
metrics = ['Precision', 'Recall', 'F1 Score', 'Accuracy']
scores = [0.43, 0.42, 0.42, 0.76]

# Create the bar chart
plt.figure(figsize=(8, 5))
plt.bar(metrics, scores, color='salmon')
plt.ylim(0, 1.1)
plt.title('Sentiment Analysis Metrics')
plt.ylabel('Score')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Save the figure

plt.tight_layout()

plt.close()



In [ ]:
from gensim.models import KeyedVectors
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# 1) Load your real Word2Vec model
w2v_model = KeyedVectors.load_word2vec_format("/content/w2v.bin", binary=True)

# 2) Embedding utility
def get_embedding(text):
    tokens = text.lower().split()
    valid_tokens = [t for t in tokens if t in w2v_model]
    if not valid_tokens:
        return np.zeros(w2v_model.vector_size)
    return np.mean([w2v_model[t] for t in valid_tokens], axis=0)

# 3) Your evaluation data (replace with your actual retrievals/relevants)
evaluation_data = [
    {
        "query": "What is the soul?",
        "top_k": {
            "BG2.12": "Never was there a time when I did not exist...",
            "BG2.47": "You have a right to perform your duty...",
            "BG3.16": "He who does not follow the cycle of sacrifice..."
        },
        "relevant": {"BG2.12"}
    },
    # … add your other queries here …
]

# 4) Compute reranked metrics
all_prec, all_rec, all_f1, all_acc = [], [], [], []

for item in evaluation_data:
    q_vec = get_embedding(item["query"])
    ids = list(item["top_k"].keys())
    vecs = np.vstack([get_embedding(text) for text in item["top_k"].values()])

    sim = cosine_similarity(q_vec.reshape(1, -1), vecs)[0]
    rerank = [ids[i] for i in np.argsort(sim)[::-1]]
    predicted = set(rerank[:len(ids)])

    relevant = item["relevant"]
    universe = list(relevant.union(predicted))
    y_true = [1 if u in relevant else 0 for u in universe]
    y_pred = [1 if u in predicted else 0 for u in universe]

    all_prec.append(precision_score(y_true, y_pred))
    all_rec.append(recall_score(y_true, y_pred))
    all_f1.append(f1_score(y_true, y_pred))
    all_acc.append(accuracy_score(y_true, y_pred))

# 5) Average and display
df_metrics_reranked = pd.DataFrame({
    "Metric": ["Precision", "Recall", "F1 Score", "Accuracy"],
    "Score": [
        np.mean(all_prec),
        np.mean(all_rec),
        np.mean(all_f1),
        np.mean(all_acc)
    ]
})
display(df_metrics_reranked)


ModuleNotFoundError: No module named 'gensim'